## Descripción del set de datos:
- id: id del vehiculo, no tiene utilidad en análsis ni modelo predictivo.
- region: Zona donde se encuentra el vehiculo, pero teniendo el estado un poco mas adelante tampoco me es de mucha utilidad.
- price: Precio de venta del vehiculo(sera la variable respuesta en los modelos predivtivos)
- year: Año de fabricación del vehiculo.
- manufacturer: Fabricante.
- model: Modelo del vehiculo.
- condition: Estado del vehiculo.
- cylinders : Número de cilindros que tiene el motor del vehiculo.
- fuel: Tipo de combustible que consume.
- odometer: Millas que tiene recorridas.
- title_status: El 95% de los datos es "clean", no es un valor relevante ni para el analisis ni para el modelo predictivo.
- transmission: Transmision que tiene el vehículo.
- VIN : Número de bastidor del vehículo.
- drive: Modo de conducción del vehiculo.
- size: Tamaño del vehículo, categorizado en full, y el 75% son nulos, por lo que no es de utilidad.
- type: Tipo de vehículo genérico.
- paint_color: Color del vehiculo.
- description: Descripcion en el anuncio.
- county: Condado, 100% nulos.
- state: Estado donde esta el vehículo.
- lat: Latitud
- long: Longitud.
- posting_date: Fecha del anuncio.

# Limpieza del set de datos
Antes de hacer un analisis de negocio, o incluso antes de realizar cualquier cálculo con un nuevo conjunto de datos, es muy importante realizar una exploración descriptiva de los mismos. Este proceso permite entender mejor qué información contiene cada variable, así como detectar posibles errores. Algunos ejemplos frecuentes son:

Que una columna se haya almacenado con el tipo incorrecto: una variable numérica está siendo reconocida como texto o viceversa.

Que una variable contenga valores que no tienen sentido: por ejemplo, para indicar que no se dispone del precio de una vivienda se introduce el valor 0 o un espacio en blanco.

Que en una variable de tipo numérico se haya introducido una palabra en lugar de un número.

Además, este análisis inicial puede dar pistas sobre qué variables son adecuadas como predictores en un modelo (más sobre esto en los siguientes apartados).

In [324]:
import pandas as pd
import numpy as np
import re
import sidetable
from scipy import stats
import math
import pickle
pd.options.display.max_columns = None

from Levenshtein import distance # Dandole unos modelos, te compara una columna y te devuelve los valores de esta transformados al mas parecido del modelo.

#librerias limpieza de nulos
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

#Normalizacion
from sklearn.preprocessing import MinMaxScaler


#Estandarización
## esto es un metodo que estandariza automaticamente todas las columnas del dataframe que le pasemos
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

#Normalizacion
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder  
from sklearn.preprocessing import OrdinalEncoder



#libreria para el balanceo
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# librerías de visualización
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt

# librerías para crear el modelo
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from xgboost import XGBRegressor
from  sklearn.ensemble import GradientBoostingRegressor


# para calcular las métricas regresion logistica
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 
from sklearn.metrics import cohen_kappa_score


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore")


In [325]:
df = pd.read_csv("datos/vehicles.csv")
df.sample(20)

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
258777,7304052366,https://southjersey.craigslist.org/ctd/d/farmi...,south jersey,https://southjersey.craigslist.org,18995,2018.0,volkswagen,tiguan,NaN,NaN,gas,9583.0,clean,automatic,3VV3B7AX9JM010070,NaN,NaN,NaN,NaN,https://images.craigslist.org/00000_edGU0tICvA...,2018 Volkswagen Tiguan CALL Automotive Avenues...,NaN,nj,40.204300,-74.177900,2021-04-09T06:09:22-0400
356093,7306580886,https://knoxville.craigslist.org/ctd/d/jacksbo...,knoxville,https://knoxville.craigslist.org,0,2018.0,acura,mdx sport hybrid 3.0l,good,6 cylinders,hybrid,81231.0,clean,automatic,5J8YD7H7XJL000395,NaN,NaN,other,NaN,https://images.craigslist.org/00z0z_3I6UTi3gdX...,Why Buy From Short-Redmond Ford?Short-Redmond ...,NaN,tn,36.353521,-84.153891,2021-04-14T07:40:54-0400
367321,7303130345,https://collegestation.craigslist.org/ctd/d/br...,college station,https://collegestation.craigslist.org,2500,1976.0,chevrolet,cheyenne,NaN,8 cylinders,gas,99999.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/00r0r_8CgEqMtEcW...,****** 1976 Chevy Square Body****** !!!!!! CA...,NaN,tx,30.820200,-96.305900,2021-04-07T10:30:30-0500
223300,7311195487,https://stjoseph.craigslist.org/cto/d/clarksda...,st joseph,https://stjoseph.craigslist.org,24800,2017.0,ford,explorer limited,excellent,6 cylinders,gas,94127.0,clean,automatic,NaN,4wd,NaN,SUV,blue,https://images.craigslist.org/00l0l_4vblTF02ge...,2017 Explorer Limited Virtually all the optio...,NaN,mo,39.798400,-94.517500,2021-04-22T21:56:17-0500
236317,7305312822,https://fayetteville.craigslist.org/ctd/d/merc...,fayetteville,https://fayetteville.craigslist.org,26998,2016.0,mercedes-benz,metris,NaN,4 cylinders,gas,100846.0,clean,automatic,NaN,rwd,NaN,van,brown,https://images.craigslist.org/00A0A_hkj7yNP1AD...,2016 Mercedes Benz Metris Conversion Van High ...,NaN,nc,NaN,NaN,2021-04-11T16:06:19-0400
361884,7312318428,https://tricities.craigslist.org/ctd/d/blountv...,tri-cities,https://tricities.craigslist.org,25990,2012.0,jeep,wrangler unlimited sahara,good,6 cylinders,gas,83995.0,clean,other,1C4BJWEG2CL120352,4wd,NaN,other,silver,https://images.craigslist.org/00B0B_4JeQXjI9PI...,Carvana is the safer way to buy a car During t...,NaN,tn,36.530000,-82.320000,2021-04-25T10:51:29-0400
124432,7309090556,https://augusta.craigslist.org/ctd/d/lexington...,augusta,https://augusta.craigslist.org,3150,2007.0,kia,NaN,excellent,NaN,gas,195.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/00g0g_2rNdCLeyJK...,"Columbia, South Carolina 2007 Kia Sportage Pr...",NaN,ga,33.952308,-81.222954,2021-04-19T00:34:39-0400
209346,7302705705,https://saginaw.craigslist.org/ctd/d/chesaning...,saginaw-midland-baycity,https://saginaw.craigslist.org,5495,2011.0,buick,regal,NaN,NaN,gas,105093.0,clean,automatic,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/00v0v_gCV5WiVSV0...,WE HAVE OVER 500 VEHICLES IN STOCK! See our ...,NaN,mi,43.182400,-84.112200,2021-04-06T13:58:00-0400
368679,7316972508,https://dallas.craigslist.org/dal/ctd/d/arling...,dallas / fort worth,https://dallas.craigslist.org,0,2015.0,ford,focus,NaN,NaN,gas,NaN,clean,manual,NaN,NaN,compact,hatchback,silver,https://images.craigslist.org/00p0p_iSM0ng8kKC...,2015 Ford Focus SE Hatch Offered by: AutoM...,NaN,tx,32.738487,-97.097679,2021-05-04T17:01:14-0500
393813,7312292704,https://norfolk.craigslist.org/ctd/d/white-mar...,norfolk / hampton roads,https://norfolk.craigslist.org,5995,2008.0,pontiac,g/6 gt,like new,6 cylinders,gas,56926.0,clean,automatic,NaN,fwd,NaN,coupe,silver,https://images.craigslist.org/00s0s_b5N6GkvPdP...,"2008 Pontiac G/6 GT Cpe, 3.5 v6 engine, Automa...",NaN,va,37.345188,-76.523316,2021-04-25T09:50:33-0400


In [326]:
df.shape

(426880, 26)

### Al acceder a los enlaces, son paginas fake. Se eliminan al no tener utilidad.

In [327]:
df.drop(["url","region_url","image_url"], axis = 1, inplace = True)

### Se eliminan el resto de columnas que no me valen.

In [328]:
df.drop(["id","region","title_status","VIN", "size", "description","county"], axis = 1, inplace= True)


In [329]:
df.sample(10)

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state,lat,long,posting_date
283109,3299,2010.0,kia,soul,NaN,NaN,gas,170000.0,manual,NaN,NaN,NaN,nh,42.887400,-71.302000,2021-05-04T08:21:25-0400
273862,21990,2005.0,ford,econoline commercial cutaway,good,8 cylinders,gas,45554.0,automatic,rwd,truck,white,ny,40.468785,-74.281707,2021-04-23T13:30:07-0400
53716,14900,2014.0,bmw,535i xdrive,excellent,6 cylinders,gas,108000.0,automatic,4wd,sedan,black,ca,38.691000,-121.748800,2021-05-01T17:06:49-0700
255093,16495,2016.0,mazda,cx-5,NaN,NaN,gas,72583.0,automatic,NaN,other,black,nj,40.923344,-74.143356,2021-04-25T17:13:01-0400
307733,30500,2019.0,rover,discovery sport,excellent,4 cylinders,gas,42000.0,automatic,4wd,SUV,red,ok,34.006100,-96.384700,2021-04-10T09:26:30-0500
162804,12995,2016.0,ford,taurus,NaN,6 cylinders,NaN,35230.0,automatic,fwd,sedan,brown,ia,41.207382,-96.023096,2021-04-19T12:20:44-0500
211558,17990,2017.0,subaru,impreza,NaN,NaN,gas,32200.0,automatic,NaN,hatchback,red,mn,46.815216,-92.178109,2021-05-01T08:41:56-0500
237374,25000,1988.0,bmw,325is,excellent,6 cylinders,gas,268500.0,manual,rwd,coupe,red,nc,36.212700,-79.989700,2021-04-23T15:22:23-0400
168986,26900,2017.0,ford,f150,NaN,NaN,gas,97623.0,automatic,4wd,pickup,blue,ks,37.646221,-98.740903,2021-04-29T09:59:10-0500
366943,6500,1992.0,jeep,wrangler,good,6 cylinders,gas,200000.0,manual,4wd,NaN,NaN,tx,26.133700,-97.644700,2021-04-15T17:17:29-0500


In [330]:
""" df.at[38697, "manufacturer"] = "bmw" """

' df.at[38697, "manufacturer"] = "bmw" '

In [331]:
df["manufacturer"].value_counts()

ford               70985
chevrolet          55064
toyota             34202
honda              21269
nissan             19067
jeep               19014
ram                18342
gmc                16785
bmw                14699
dodge              13707
mercedes-benz      11817
hyundai            10338
subaru              9495
volkswagen          9345
kia                 8457
lexus               8200
audi                7573
cadillac            6953
chrysler            6031
acura               5978
buick               5501
mazda               5427
infiniti            4802
lincoln             4220
volvo               3374
mitsubishi          3292
mini                2376
pontiac             2288
rover               2113
jaguar              1946
porsche             1384
mercury             1184
saturn              1090
alfa-romeo           897
tesla                868
fiat                 792
harley-davidson      153
ferrari               95
datsun                63
aston-martin          24


In [332]:
df["condition"].value_counts()

good         121456
excellent    101467
like new      21178
fair           6769
new            1305
salvage         601
Name: condition, dtype: int64

In [333]:
df["transmission"].value_counts()

automatic    336524
other         62682
manual        25118
Name: transmission, dtype: int64

In [334]:
df["drive"].value_counts()

4wd    131904
fwd    105517
rwd     58892
Name: drive, dtype: int64

In [335]:
df["type"].value_counts()

sedan          87056
SUV            77284
pickup         43510
truck          35279
other          22110
coupe          19204
hatchback      16598
wagon          10751
van             8548
convertible     7731
mini-van        4825
offroad          609
bus              517
Name: type, dtype: int64

In [336]:
#se convierte los prefijos de los estados a mayusculas.
df["state"] = df["state"].str.upper()
df["state"].value_counts()

CA    50614
FL    28511
TX    22945
NY    19386
OH    17696
OR    17104
MI    16900
NC    15277
WA    13861
PA    13753
WI    11398
CO    11088
TN    11066
VA    10732
IL    10387
NJ     9742
ID     8961
AZ     8679
IA     8632
MA     8174
MN     7716
GA     7003
OK     6792
SC     6327
MT     6294
KS     6209
IN     5704
CT     5188
AL     4955
MD     4778
NM     4425
MO     4293
KY     4149
AR     4038
AK     3474
LA     3196
NV     3194
NH     2981
DC     2970
ME     2966
HI     2964
VT     2513
RI     2320
SD     1302
UT     1150
WV     1052
NE     1036
MS     1016
DE      949
WY      610
ND      410
Name: state, dtype: int64

In [337]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         426880 non-null  int64  
 1   year          425675 non-null  float64
 2   manufacturer  409234 non-null  object 
 3   model         421603 non-null  object 
 4   condition     252776 non-null  object 
 5   cylinders     249202 non-null  object 
 6   fuel          423867 non-null  object 
 7   odometer      422480 non-null  float64
 8   transmission  424324 non-null  object 
 9   drive         296313 non-null  object 
 10  type          334022 non-null  object 
 11  paint_color   296677 non-null  object 
 12  state         426880 non-null  object 
 13  lat           420331 non-null  float64
 14  long          420331 non-null  float64
 15  posting_date  426812 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 52.1+ MB


### Para e tratamiento de nulos, los voy a separar en 2 partes, la visualizacion y machine learning. Para la visualizacion voy a crear la variable desconocida, y eliminar el menor numero de datos.Para el modelo al ser la mayoria variables categoricas, voy a realizar primero el encoding de estas. Se realiza el encoding para poder aplicar el método KNN imputer de reemplazar los nulos por valores similiares al de sus vecinos mas parecidos.

In [338]:
df.isnull().sum()/df.shape[0]*100

price            0.000000
year             0.282281
manufacturer     4.133714
model            1.236179
condition       40.785232
cylinders       41.622470
fuel             0.705819
odometer         1.030735
transmission     0.598763
drive           30.586347
type            21.752717
paint_color     30.501078
state            0.000000
lat              1.534155
long             1.534155
posting_date     0.015930
dtype: float64

### Al ser un set de datos de vehiculos,lo mas normal es que haya vehiculos repetidos.Usando esa ventaja, se sustituyen los valores nulos de algunas columnas, solo si coincide el modelo, por el valor que le corresponde.

In [339]:
df.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state,lat,long,posting_date
0,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AZ,NaN,NaN,NaN
1,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AR,NaN,NaN,NaN
2,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL,NaN,NaN,NaN
3,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MA,NaN,NaN,NaN
4,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,NaN,NaN,NaN


In [340]:
df2 = df.copy()
df2.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state,lat,long,posting_date
0,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AZ,NaN,NaN,NaN
1,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AR,NaN,NaN,NaN
2,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FL,NaN,NaN,NaN
3,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MA,NaN,NaN,NaN
4,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NC,NaN,NaN,NaN


In [341]:
# En este caso, se rellenan los valores nulos de la columna manufacturer.
# Se crea un diccionario con los valores a reemplazar
reemplazos = dict(df.groupby('model')['manufacturer'].apply(lambda x: x.ffill().bfill()))

# Se usa el diccionario para reemplazar los nulos en la columna "manufacturer"
df['manufacturer'] = df['manufacturer'].fillna(reemplazos)


In [342]:
reemplazos

{27: 'gmc',
 28: 'chevrolet',
 29: 'chevrolet',
 30: 'toyota',
 31: 'ford',
 32: 'gmc',
 33: 'chevrolet',
 34: 'toyota',
 35: 'chevrolet',
 36: 'chevrolet',
 37: 'jeep',
 38: 'jeep',
 39: 'chevrolet',
 40: 'chevrolet',
 41: 'toyota',
 42: 'chevrolet',
 43: 'toyota',
 44: 'ford',
 45: 'nissan',
 46: 'jeep',
 47: 'ford',
 48: 'toyota',
 49: 'jeep',
 50: 'ford',
 51: 'ram',
 52: 'mazda',
 53: 'ford',
 54: 'cadillac',
 55: 'ford',
 56: 'jeep',
 57: 'ford',
 58: 'gmc',
 59: 'honda',
 60: 'ram',
 61: 'gmc',
 62: 'ram',
 63: 'ford',
 64: 'chevrolet',
 65: 'ford',
 66: 'chevrolet',
 67: 'toyota',
 68: 'chevrolet',
 69: 'chevrolet',
 70: 'toyota',
 71: 'chevrolet',
 72: 'jeep',
 73: 'dodge',
 74: 'honda',
 75: 'honda',
 76: 'ford',
 77: 'honda',
 78: 'ford',
 79: 'jeep',
 80: 'toyota',
 81: 'ford',
 82: 'lexus',
 83: 'jaguar',
 84: 'buick',
 85: 'lexus',
 86: 'chrysler',
 87: 'volvo',
 88: 'volvo',
 89: 'volvo',
 90: 'volvo',
 91: 'audi',
 92: 'infiniti',
 93: 'lincoln',
 94: 'alfa-romeo',
 95:

In [343]:
# Se rellenan los valores nulos de la columna cylinders.
# Se crea un diccionario con los valores a reemplazar
reemplazos = dict(df.groupby('model')['cylinders'].apply(lambda x: x.ffill().bfill()))

# Se usa el diccionario para reemplazar los nulos en la columna "manufacturer"
df['cylinders'] = df['cylinders'].fillna(reemplazos)

In [344]:
""" Al realizar esto, se han sustituido un 32% de los valores nulos que habia"""
df.isnull().sum()/df.shape[0]*100

price            0.000000
year             0.282281
manufacturer     3.953804
model            1.236179
condition       40.785232
cylinders        9.525628
fuel             0.705819
odometer         1.030735
transmission     0.598763
drive           30.586347
type            21.752717
paint_color     30.501078
state            0.000000
lat              1.534155
long             1.534155
posting_date     0.015930
dtype: float64

In [345]:
# Se rellenan los valores nulos de la columna drive.
# Se crea un diccionario con los valores a reemplazar
reemplazos = dict(df.groupby('model')['drive'].apply(lambda x: x.ffill().bfill()))

# Se usa el diccionario para reemplazar los nulos en la columna "manufacturer"
df['drive'] = df['drive'].fillna(reemplazos)

In [346]:
"""Se ha reemplazado un 24% de los valores nulos, por sus valores correspondientes, teniendo en cuenta el modelo. """
df.isnull().sum()/df.shape[0]*100

price            0.000000
year             0.282281
manufacturer     3.953804
model            1.236179
condition       40.785232
cylinders        9.525628
fuel             0.705819
odometer         1.030735
transmission     0.598763
drive            6.083443
type            21.752717
paint_color     30.501078
state            0.000000
lat              1.534155
long             1.534155
posting_date     0.015930
dtype: float64

In [347]:
# Se rellenan los valores nulos de la columna type.
# Se crea un diccionario con los valores a reemplazar
reemplazos = dict(df.groupby('model')['type'].apply(lambda x: x.ffill().bfill()))

# Se usa el diccionario para reemplazar los nulos en la columna "manufacturer"
df['type'] = df['type'].fillna(reemplazos)

In [348]:
"""Se ha reemplazado un 19% de los valores nulos"""
df.isnull().sum()/df.shape[0]*100

price            0.000000
year             0.282281
manufacturer     3.953804
model            1.236179
condition       40.785232
cylinders        9.525628
fuel             0.705819
odometer         1.030735
transmission     0.598763
drive            6.083443
type             2.954226
paint_color     30.501078
state            0.000000
lat              1.534155
long             1.534155
posting_date     0.015930
dtype: float64

In [349]:
# Se rellenan los valores nulos de la columna paint_color.
# Se crea un diccionario con los valores a reemplazar
reemplazos = dict(df.groupby('model')['paint_color'].apply(lambda x: x.ffill().bfill()))

# Se usa el diccionario para reemplazar los nulos en la columna "manufacturer"
df['paint_color'] = df['paint_color'].fillna(reemplazos)

In [350]:
# Se rellenan los valores nulos de la columna lat, en este caso la referencia es el estado.
# Se crea un diccionario con los valores a reemplazar
reemplazos = dict(df.groupby('state')['lat'].apply(lambda x: x.ffill().bfill()))

# Se usa el diccionario para reemplazar los nulos en la columna "manufacturer"
df['lat'] = df['lat'].fillna(reemplazos)

In [351]:
# Se rellenan los valores nulos de la columna long, en este caso la referencia es el estado.
# Se crea un diccionario con los valores a reemplazar
reemplazos = dict(df.groupby('state')['long'].apply(lambda x: x.ffill().bfill()))

# Se usa el diccionario para reemplazar los nulos en la columna "manufacturer"
df['long'] = df['long'].fillna(reemplazos)

In [352]:
# Se rellenan los valores nulos de la columna fuel.
# Se crea un diccionario con los valores a reemplazar
reemplazos = dict(df.groupby('model')['fuel'].apply(lambda x: x.ffill().bfill()))

# Se usa el diccionario para reemplazar los nulos en la columna "manufacturer"
df['fuel'] = df['fuel'].fillna(reemplazos)

In [353]:
df.isnull().sum()/df.shape[0]*100

price            0.000000
year             0.282281
manufacturer     3.953804
model            1.236179
condition       40.785232
cylinders        9.525628
fuel             0.371533
odometer         1.030735
transmission     0.598763
drive            6.083443
type             2.954226
paint_color      4.675319
state            0.000000
lat              0.000000
long             0.000000
posting_date     0.015930
dtype: float64

In [354]:
#Se elimina la palabra "cylinders" de la columna, dejandola asi solo un numero como valor.
df["cylinders"] = df["cylinders"].str.split(' ').str[0]

### Después de aplicar este método, hay un set de datos bastante mas limpio. Ahora se prodece a eliminar las filas que tengan mas de 4 valores nulos, ya que contienen poca información útil.

In [355]:
""" Elimino los valores que tengas mas de 5 valores nulos en sus datos
el parametro tresh filtra y elimina las filas con el minimo de valores correctos, tenemos que restarle los nulos maximos que queramos
    """
df2=df.copy()
df2.dropna(thresh=df.shape[1]- 5, inplace= True)
df2.isnull().sum()/df2.shape[0]*100


price            0.000000
year             0.265484
manufacturer     3.571412
model            0.918221
condition       40.351949
cylinders        8.859149
fuel             0.335572
odometer         0.997749
transmission     0.563535
drive            5.391572
type             2.256970
paint_color      3.977780
state            0.000000
lat              0.000000
long             0.000000
posting_date     0.000000
dtype: float64

In [356]:
print(df2.shape, df.shape)

(423754, 16) (426880, 16)


### De esta manera se han eliminado un 0.1% de los datos para su visualización, dejando una muestra significativa para su estudio.

### El resto de los nulos se sustituiran por el valor "desconocido".

### La variable condición, no se puede sustituir por valores similares para su visualización, ya que seria "inventarse" los datos. Para el modelo predictivo si se puede hacer esa sustitución.

In [357]:
df2.fillna("desconocido",inplace=True)

In [358]:
df2.isnull().sum()/df2.shape[0]*100

price           0.0
year            0.0
manufacturer    0.0
model           0.0
condition       0.0
cylinders       0.0
fuel            0.0
odometer        0.0
transmission    0.0
drive           0.0
type            0.0
paint_color     0.0
state           0.0
lat             0.0
long            0.0
posting_date    0.0
dtype: float64

In [359]:
df2.replace(to_replace="other",value="desconocido", inplace=True)

In [360]:
df2.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state,lat,long,posting_date
27,33590,2014.0,gmc,sierra 1500 crew cab slt,good,8,gas,57923.0,desconocido,4wd,pickup,white,AL,32.590,-85.4800,2021-05-04T12:31:18-0500
28,22590,2010.0,chevrolet,silverado 1500,good,8,gas,71229.0,desconocido,4wd,pickup,blue,AL,32.590,-85.4800,2021-05-04T12:31:08-0500
29,39590,2020.0,chevrolet,silverado 1500 crew,good,8,gas,19160.0,desconocido,4wd,pickup,red,AL,32.590,-85.4800,2021-05-04T12:31:25-0500
30,30990,2017.0,toyota,tundra double cab sr,good,8,gas,41124.0,desconocido,4wd,pickup,red,AL,32.590,-85.4800,2021-05-04T10:41:31-0500
31,15000,2013.0,ford,f-150 xlt,excellent,6,gas,128000.0,automatic,rwd,truck,black,AL,32.592,-85.5189,2021-05-03T14:02:03-0500


In [361]:
#Se elimina esta columna porque todos los anuncios son en el mes de abril y mayo.
df2.drop("posting_date", axis = 1, inplace=True)

### Ahora toca limpiar la columna de modelo, cada fabricante tiene sus modelos y estos escritos de manera distinta, asi hay que mirarlos para corregirlos.

In [362]:
marcas = df2["manufacturer"].unique().tolist()
#43 marcas

In [363]:
def formato_texto(texto):
    # Reemplazar los guiones por espacios vacíos
    texto = texto.replace("-", " ")
    
    # Convertir el texto a título (primera letra de cada palabra en mayúscula)
    texto = texto.title()
    
    # Reemplazar los espacios por guiones bajos
    texto = texto.replace(" ", "_")
    
    # Devolver el texto formateado
    return texto


In [364]:
df2["manufacturer"] = df2["manufacturer"].apply(formato_texto)

In [365]:
df2["manufacturer"].value_counts()

Ford               70813
Chevrolet          54897
Toyota             34372
Honda              21219
Nissan             19032
Jeep               19001
Ram                18251
Gmc                16730
Desconocido        15134
Bmw                14693
Dodge              13651
Mercedes_Benz      11800
Hyundai            10327
Subaru              9484
Volkswagen          9324
Kia                 8444
Lexus               8182
Audi                7569
Cadillac            6926
Chrysler            6035
Acura               5965
Buick               5495
Mazda               5446
Infiniti            4796
Lincoln             4204
Volvo               3311
Mitsubishi          3289
Mini                2374
Pontiac             2280
Rover               2141
Jaguar              1939
Porsche             1379
Mercury             1183
Saturn              1085
Alfa_Romeo           897
Tesla                867
Fiat                 858
Harley_Davidson      151
Ferrari               95
Datsun                68


#### gmc

In [366]:
gmc = df2[df2["manufacturer"] == "Gmc"]
# Parece que en este fabricante el modelo es la primera palabra y lo siguiente es descripcion del modelo, asique solo interesa la primera
#gmc.sample(20)

#### Vemos los valores unicos de cada modelo para plantear el problema.

#### Con ayuda de chatGPT generamos una lista de todos los modelos de la marca gmc:
####   models = ['Canyon', 'Sierra 1500', 'Sierra 2500HD', 'Sierra 3500HD', 'Sierra Denali', 'Sierra AT4', 'Acadia', 'Terrain', 'Yukon', 'Yukon XL', 'Yukon Denali', "Jimmy", "Safari","Suburban",'Savana', "Brigadier", "Envoy","Graphyte", "Granite", "Sonoma","Syclone", "TopKick","Vandura",'TopKick C4500', 'TopKick C5500', 'TopKick C6500', 'TopKick C7500', "Tracker","Typhoon"]


In [367]:
"""## Con el metodo Levenshtein, teniendo la lista de modelos reales nos devuelve, transformados a los mas
## parecidos de la lista de modelos reales.

def find_closest_model(input_model):
    models = ['Canyon', 'Sierra 1500', 'Sierra 2500HD', 'Sierra 3500HD', 'Sierra Denali', 'Sierra AT4', 
              'Acadia', 'Terrain', 'Yukon', 'Yukon XL', 'Yukon Denali', "Jimmy", "Safari","Suburban",
              'Savana', "Brigadier", "Envoy","Graphyte", "Granite", "Sonoma","Syclone", "TopKick","Vandura",
              'TopKick C4500', 'TopKick C5500', 'TopKick C6500', 'TopKick C7500', "Tracker","Typhoon"]
    closest_match = min(models, key=lambda x: distance(x.lower(), input_model.lower()))
    return closest_match


# aplicar la función a la columna 'model' y guardar el resultado en una nueva columna 'modelo'
gmc['model'] = gmc['model'].apply(find_closest_model)"""




'## Con el metodo Levenshtein, teniendo la lista de modelos reales nos devuelve, transformados a los mas\n## parecidos de la lista de modelos reales.\n\ndef find_closest_model(input_model):\n    models = [\'Canyon\', \'Sierra 1500\', \'Sierra 2500HD\', \'Sierra 3500HD\', \'Sierra Denali\', \'Sierra AT4\', \n              \'Acadia\', \'Terrain\', \'Yukon\', \'Yukon XL\', \'Yukon Denali\', "Jimmy", "Safari","Suburban",\n              \'Savana\', "Brigadier", "Envoy","Graphyte", "Granite", "Sonoma","Syclone", "TopKick","Vandura",\n              \'TopKick C4500\', \'TopKick C5500\', \'TopKick C6500\', \'TopKick C7500\', "Tracker","Typhoon"]\n    closest_match = min(models, key=lambda x: distance(x.lower(), input_model.lower()))\n    return closest_match\n\n\n# aplicar la función a la columna \'model\' y guardar el resultado en una nueva columna \'modelo\'\ngmc[\'model\'] = gmc[\'model\'].apply(find_closest_model)'

In [368]:
gmc["model"].value_counts()

sierra 1500                1657
sierra                     1282
acadia                      925
sierra 2500hd               895
yukon                       884
                           ... 
sierra sle 2500hd             1
terrian sle2                  1
sierra extended cab 4x4       1
3500 hd sl                    1
sevana (custom)               1
Name: model, Length: 1173, dtype: int64

### Añadimos estos valores al set de datos original(df2)

In [369]:
df2["model"][df2["manufacturer"]== "gmc"] = gmc["model"]

## Se hace lo mismo con el resto de marcas

### Se definen las listas de modelos segun las marcas.

In [370]:
dicc_marcas = { "Gmc" : ['Canyon', 'Sierra 1500', 'Sierra 2500HD', 'Sierra 3500HD', 'Sierra Denali', 'Sierra AT4', 'Acadia', 'Terrain', 'Yukon', 'Yukon XL', 'Yukon Denali', "Jimmy", "Safari","Suburban",'Savana', "Brigadier", "Envoy","Graphyte", "Granite", "Sonoma","Syclone", "TopKick","Vandura",'TopKick C4500', 'TopKick C5500', 'TopKick C6500', 'TopKick C7500', "Tracker","Typhoon"]
    ,"Chevrolet" : ["Camaro","Caprice","Cavalier","Chevelle","Cobalt","Colorado","Corvette","Cruze","El Camino","Equinox","Express","HHR","Impala","Malibu","Monte Carlo","Silverado","Sonic","Spark","Suburban","Tahoe","Tracker","Trailblazer","Traverse"]

,"Toyota" : ["4Runner","Avalon", "Camry","Celica","Corolla","Echo","FJ Cruiser","Highlander","Land Cruiser","Matrix","MR2","Prius","RAV4","Sequoia","Sienna","Supra","Tacoma","Tundra","Venza","Yaris"]

,"Ford" : ["Aerostar","Bronco","Crown Victoria","Edge","Escape","Escort","Excursion","Expedition","Explorer","F-150","F-250","F-350","Festiva","Fiesta","Five Hundred","Flex","Focus","Fusion","Galaxie","Mustang","Probe","Ranger","Taurus","Tempo","Thunderbird","Transit","Windstar"]

,"Jeep" : ["Cherokee","Commander","Compass","Gladiator","Grand Cherokee","Liberty","Patriot","Renegade","Wrangler"]

,"Nissan" : ["240SX","300ZX","350Z","Altima","Armada","Cube","Frontier","GT-R","Juke","Leaf","Maxima","Murano","NV","Pajero","Pathfinder","Quest","Rogue","Sentra","Stanza","Titan","Versa","Xterra" ]

,"Ram" : ["1500","2500","3500","Cargo Van","ProMaster"]

,"Mazda" : ["323","626","B-Series","CX-3","CX-5","CX-7","CX-9","Miata","MPV","MX-3","MX-5","Navajo","Protege","RX-7","RX-8","Tribute" ]

,"Cadillac" : ["ATS","CT6","CTS","DeVille","DTS","Eldorado","Escalade","Fleetwood","Seville","SRX","STS","XLR" ]

,"Honda" : ["Accord","Civic","Clarity","CR-V","CR-X","Crosstour","Element","Fit","HR-V","Insight","Odyssey","Passport","Pilot","Prelude","Ridgeline","S2000" ]

,"Dodge" : ["Avenger","Caliber","Caravan","Challenger","Charger","Dart","Durango","Grand Caravan","Journey","Magnum","Neon","Nitro","Ram","Stratus","Viper"]

,"Lexus" : ["ES","GS","GX","IS","LC","LS","LX","NX","RC","RX","SC" ]

,"Jaguar" : [ "E-PACE","F-PACE","F-TYPE","S-TYPE","X-TYPE","XE","XF","XJ","XK"]

,"Buick" :["Century","Enclave","Encore","LaCrosse","LeSabre","Lucerne","Park Avenue","Rainier","Regal","Rendezvous","Riviera","Roadmaster","Skylark","Terraza","Verano"]

,"Chrysler" : ["200","300","Aspen","Crossfire","Pacifica","PT Cruiser","Sebring","Town and Country"]

,"Volvo" : ["240","740","760","850","940","C30","C70","S40","S60","S70","S80","V40","V50","V60","V70","XC40","XC60","XC70","XC90" ]

,"Audi" : ["100","200","A3","A4","A5","A6","A7","A8","Allroad","Q3","Q5","Q7","R8","RS3","RS4","RS5","RS6","S3","S4","S5","S6","S7","S8",'TT' ]

,"Infiniti" : [ "EX","FX","G20","G25","G35","G37","I30","I35","J30","M30","M35","M37","M45","Q30","Q40","Q45","Q50","Q60","Q70","QX30","QX4","QX50","QX56","Q"]

,"Lincoln" : ["Aviator", "Continental", "Corsair", "MKC", "MKT", "MKX", "MKZ", "Nautilus", "Navigator"]

,"Alfa_Romeo" : ["4C", "Giulia", "Mito", "Stelvio", "Spider"]

,"Subaru" : ["Ascent", "BRZ", "Crosstrek", "Forester", "Impreza", "Legacy", "Outback", "WRX"]

,"Acura" : ["ILX", "MDX", "NSX", "RDX", "RLX", "TLX"]

,"Hyundai" : ["Accent", "Elantra", "Genesis","h1", "i10", "i20","i30","i40", "Kona", "Palisade", "Santa Fe", "Sonata", "Tucson", "Veloster"]

,"Mercedes_Benz" :  ["A-Class", "C-Class", "E-Class", "S-Class", "GLE", "GLC", "GLA", "GLS","SL","AMG GT"]

,"Bmw" : ["2 Series", "3 Series", "4 Series", "5 Series", "7 Series", "X1", "X2", "X3", "X4", "X5", "X6", "X7", "Z3", "Z4"]

,"Mitsubishi" : ["ASX","Colt","Eclipse" ,"EVO","Galant", "Lancer","L200","Mirage", "Mirage G4","Montero", "Outlander", "Outlander PHEV","Space"]

,"Volkswagen" : ["Arteon", "Atlas", "Beetle","California", "Golf", "Jetta", "Scirocco","Passat", "Polo","Tiguan","Touran" "T-Roc"]

,"Porsche" : ["718", "911","Boxter","Cayman", "Cayenne","GT3", "Macan", "Panamera", "Taycan" ]

,"Kia" : ["Forte", "Niro", "Optima", "Rio", "Sedona", "Sorento", "Soul", "Sportage", "Stinger", "Telluride"]

,"Rover" : ["25","45", "75", "200", "220"]

,"Ferrari" : ["488", "812 Superfast", "California T", "F8 Spider", "GTC4Lusso", "Portofino", "Roma", "SF90 Stradale"]

,"Mini" : ["Clubman", "Cabrio", "Countryman", "Hardtop 2 Door", "Hardtop 4 Door" , "One"]

,"Pontiac" : [ "Aztek", "Fiero", "Firebird", "G6", "GTO", "Solstice", "Sunfire"]

,"Fiat" : ["Multipla","Doblo", "Ducato", "Panda", "Punto","124 Spider", "500", "500X", "500L"]

,"Tesla" :["Model S", "Model X", "Model 3", "Model Y", "Roadster"]

,"Saturn" :["Aura", "Ion", "Outlook", "Sky", "Vue"]

,"Mercury" : ["Grand Marquis", "Marauder", "Milan", "Montego", "Monterey", "Mountaineer", "Sable"]

,"Aston_Martin": ["DB9","DB11", "DBS Superleggera","DBX", "Rapide AMR", "Vantage", "Vanquish"]

,"Land_Rover": ["Defender", "Discovery","Evoque", "Sport", "Velar"]

,"Morgan" :["3 Wheeler", "Plus Four", "Roadster", "Plus Six"]}

In [371]:
df3 = df2.copy()

In [372]:
def modelo_cercano(marca, modelo, diccionario):
    try:
        models = diccionario[marca]
        closest_match = min(models, key=lambda x: distance(x, modelo.lower()))
        return closest_match
    except:
        return modelo


df2["model"] = df2.apply(lambda dataframe: modelo_cercano(dataframe["manufacturer"], dataframe["model"], dicc_marcas), axis = 1) 

In [373]:
df2.head()

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state,lat,long
27,33590,2014.0,Gmc,Sierra 1500,good,8,gas,57923.0,desconocido,4wd,pickup,white,AL,32.590,-85.4800
28,22590,2010.0,Chevrolet,Silverado,good,8,gas,71229.0,desconocido,4wd,pickup,blue,AL,32.590,-85.4800
29,39590,2020.0,Chevrolet,Silverado,good,8,gas,19160.0,desconocido,4wd,pickup,red,AL,32.590,-85.4800
30,30990,2017.0,Toyota,Land Cruiser,good,8,gas,41124.0,desconocido,4wd,pickup,red,AL,32.590,-85.4800
31,15000,2013.0,Ford,F-150,excellent,6,gas,128000.0,automatic,rwd,truck,black,AL,32.592,-85.5189


In [374]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423754 entries, 27 to 426879
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         423754 non-null  int64  
 1   year          423754 non-null  object 
 2   manufacturer  423754 non-null  object 
 3   model         423754 non-null  object 
 4   condition     423754 non-null  object 
 5   cylinders     423754 non-null  object 
 6   fuel          423754 non-null  object 
 7   odometer      423754 non-null  object 
 8   transmission  423754 non-null  object 
 9   drive         423754 non-null  object 
 10  type          423754 non-null  object 
 11  paint_color   423754 non-null  object 
 12  state         423754 non-null  object 
 13  lat           423754 non-null  float64
 14  long          423754 non-null  float64
dtypes: float64(2), int64(1), object(12)
memory usage: 67.9+ MB


In [375]:
df2[df2["manufacturer"]== "Desconocido"]

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state,lat,long
97,15990,2016.0,Desconocido,Scion iM Hatchback 4D,good,desconocido,gas,29652.0,desconocido,fwd,hatchback,blue,AL,32.5900,-85.4800
122,6800,2005.0,Desconocido,blue bird bus,excellent,6,diesel,180000.0,automatic,rwd,bus,yellow,AL,32.8337,-85.1197
135,14990,2016.0,Desconocido,Scion iM Hatchback 4D,good,desconocido,gas,65203.0,desconocido,fwd,hatchback,red,AL,32.5900,-85.4800
137,2500,1966.0,Desconocido,1966 C-30 1 ton,good,6,gas,47000.0,manual,rwd,pickup,brown,AL,32.4169,-85.7094
155,8990,2013.0,Desconocido,smart fortwo Passion Hatchback,good,desconocido,gas,59072.0,automatic,rwd,coupe,silver,AL,32.5900,-85.4800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426843,21999,2017.0,Desconocido,2017,desconocido,4,gas,51000.0,automatic,4wd,sedan,black,WY,48.1783,-111.9397
426846,41999,2015.0,Desconocido,2015,desconocido,4,gas,126000.0,automatic,4wd,sedan,blue,WY,48.1783,-111.9397
426862,27590,2018.0,Desconocido,Genesis G80 3.8 Sedan 4D,good,6,gas,36138.0,desconocido,rwd,sedan,black,WY,33.7865,-84.4454
426863,25590,2017.0,Desconocido,Genesis G80 3.8 Sedan 4D,good,6,gas,37608.0,automatic,rwd,sedan,white,WY,33.7865,-84.4454


In [376]:
df2["model"][df2["manufacturer"]== "Datsun"]
df2["model"][df2["manufacturer"]== "Datsun"] = df2["model"][df2["manufacturer"]== "Datsun"].str.split(' ', expand=True)[0]




#### Harley es una marca de motos, y esto es un analisis de coches, asique se elimina del set de datos.

In [377]:
df2.drop(df2.index[df2["manufacturer"] == "Harley_Davidson"], axis = 0, inplace=True)

In [378]:
df2[df2["price"] == 0]

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color,state,lat,long
46,0,2011.0,Jeep,Compass,excellent,4,gas,99615.0,automatic,fwd,SUV,white,AL,32.547500,-85.468200
126,0,2018.0,Chevrolet,Express,like new,6,gas,68472.0,automatic,rwd,van,white,AL,31.425091,-85.617723
127,0,2019.0,Chevrolet,Express,like new,6,gas,69125.0,automatic,rwd,van,white,AL,31.425091,-85.617723
128,0,2018.0,Chevrolet,Express,like new,6,gas,66555.0,automatic,rwd,van,white,AL,31.425091,-85.617723
191,0,2015.0,Nissan,Sentra,excellent,4,gas,99505.0,automatic,fwd,sedan,silver,AL,33.614088,-85.796729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426764,0,2018.0,Desconocido,peterbilt 579,desconocido,desconocido,diesel,1.0,automatic,rwd,truck,white,WY,42.862274,-106.307144
426812,0,2006.0,Toyota,Echo,excellent,4,gas,195000.0,automatic,fwd,hatchback,silver,WY,40.114685,-104.971971
426832,0,2004.0,Toyota,Prius,excellent,4,hybrid,239000.0,automatic,fwd,hatchback,blue,WY,40.114685,-104.971971
426836,0,2018.0,Ram,2500,excellent,6,diesel,20492.0,automatic,4wd,truck,white,WY,45.082436,-94.334686


In [379]:
df2.to_csv("datos/set_limpio.csv")

Eliminamos los precios que son igual a 0, ya que para el analisis son son utiles. Para el modelo predictivo se sustituiran.

In [380]:
df2.drop(df2.index[df2["price"] == 0], axis = 0, inplace=True)

## Ya estan los datos limpios para proceder con el eda.

In [381]:
df2.to_csv("datos/set_limpio_sin0.csv")

In [253]:
df2.isnull().sum()

price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
drive           0
type            0
paint_color     0
state           0
lat             0
long            0
dtype: int64